# Explore and Model StatFox Matchup Data
`mlb_bet_notebooks/model_statfox_matchups.ipynb`
- Explore features
- Convert historical moneylines to break-even probabilities
- Model pre-computed features with RF and maybe PCA
- Compare model predictions to historical moneylines
    - Use break-even probabilities as alternative model and compare ROC
- Try VIF filter
- Try k-fold CV
- Try grid search model complexity
- Try to get player salary
    - Combine with addition, subtraction from statfox blobs
- Try fix Opening Line feature 
    - Try openline probability as feature

Jonathan Sims 2020-02-24

In [1]:
import math
import boto3
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import roc_auc_score, make_scorer
import numpy as np
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import random
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut

### Import model data

In [2]:
s3 = boto3.client('s3')

In [3]:
df_feat_fill = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_features.tsv.gz', sep='\t', index_col=0)
df_targ = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_target.tsv.gz', sep='\t', index_col=0, header=None, squeeze=True)
df_targ_wt = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_target_weight.tsv.gz', sep='\t', index_col=0, header=None, squeeze=True)
df_lateline_prob = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_lateline_prob.tsv.gz', sep='\t', index_col=0, header=None, squeeze=True)

In [4]:
df_targ.shape

(17573,)

In [5]:
df_feat_fill.shape

(17573, 1128)

In [6]:
df_targ_wt.head()

0
0    1.769231
1    1.606061
2    2.050000
3    1.833333
4    1.769231
Name: 1, dtype: float64

In [7]:
df_targ_wt.sum()

34455.36163974437

In [8]:
### Make sure matchidx column exists

[col for col in df_feat_fill.columns if 'match' in col]

['matchidx']

In [9]:
### Partial header names to looks for unknown/erroneous columns

f = lambda x: x[2:13]
colstrip = pd.Series(df_feat_fill.columns).map(f)
[col for col in colstrip.drop_duplicates() if '_h_' not in col and '_v_' not in col]

['named: 0.1',
 'Bullpen_BB_',
 'Bullpen_BSV',
 'Bullpen_ERA',
 'Bullpen_ER_',
 'Bullpen_HR_',
 'Bullpen_H_A',
 'Bullpen_H_H',
 'Bullpen_IP_',
 'Bullpen_L_A',
 'Bullpen_L_H',
 'Bullpen_R_A',
 'Bullpen_R_H',
 'Bullpen_SO_',
 'Bullpen_SV_',
 'Bullpen_WHI',
 'Bullpen_W_A',
 'Bullpen_W_H',
 'HitField_Te',
 'Overall_Opp',
 'Overall_Tea',
 'Bullpen_H_R',
 'Bullpen_L_R',
 'Bullpen_R_R',
 'Bullpen_W_R',
 'tchidx',
 'Bullpen_Pct',
 '_Opening_Li',
 'nth',
 'ar',
 '_Latest_Tot',
 '_Opening_To']

### Make custom score function

In [11]:
def test_score(y, y_pred, wt):
    y_diff = (-1 * abs(y - y_pred) + 1 ) * wt - 1
    y_diff_sum = y_diff.sum()
    return y_diff_sum

In [12]:
sum_score = make_scorer(test_score, wt=pd.Series(df_targ_wt.values))

In [13]:
sum_score

make_scorer(test_score, wt=0        1.769231
1        1.606061
2        2.050000
3        1.833333
4        1.769231
5        1.606061
6        1.769231
7        1.714286
8        1.833333
9        2.200000
10       1.800000
11       2.350000
12       1.571429
13       2.050000
14       2.250000
15       1.714286
16       1.606061
17       2.500000
18       2.250000
19       1.769231
20       2.200000
21       2.250000
22       2.550000
23       2.100000
24       1.909091
25       1.540541
26       2.500000
27       1.833333
28       1.625000
29       1.769231
           ...   
17543    1.392157
17544    1.571429
17545    1.645161
17546    1.740741
17547    1.425532
17548    2.200000
17549    1.384615
17550    1.434783
17551    2.820000
17552    3.050000
17553    1.740741
17554    1.487805
17555    1.425532
17556    1.769231
17557    2.200000
17558    1.588235
17559    1.384615
17560    1.425532
17561    2.250000
17562    2.150000
17563    1.400000
17564    1.909091
17565    3.150000
1

In [27]:
prof_score = make_scorer(test_score, wt=df_targ_wt)
grid = GridSearchCV(RandomForestClassifier(n_jobs=-1), param_grid={'n_estimators':[200]}, scoring=prof_score)
grid.fit(df_feat_fill[:14000], df_targ[:14000])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [200]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring=make_scorer(test_score, wt=0
0        1.769231
1        1.606061
2        2.050000
3        1.833333
4        1.769231
5        1.606061
6        1.769231
7        1.714286
8        1.833333
9        2.200000
10       1.800000
11       2.350000
12       1.571429
13       2.050000
14       2....606061
17570    1.625000

In [28]:
grid.score(df_feat_fill[14000:], df_targ[14000:])

-61.184986245215356

### Calculate close moneyline ROC AUC

In [ ]:
df_targ_dropna = df_targ[df_lateline_prob.isna() == False]
df_lateline_prob_dropna = df_lateline_prob[df_lateline_prob.isna() == False]
df_targ_wt_dropna = df_targ_wt[df_lateline_prob.isna() == False]

In [ ]:
roc_auc_score(df_targ_dropna, df_lateline_prob_dropna, sample_weight=df_targ_wt_dropna)

In [ ]:
df_targ.shape

In [ ]:
df_feat_fill.shape

In [ ]:
df_targ_wt.shape

In [ ]:
df_targ_wt.values

In [ ]:
for cnt in range(0,10):

    df_feat_fill_train, df_feat_fill_test, df_targ_train, df_targ_test, df_targ_wt_train, df_targ_wt_test = train_test_split(df_feat_fill, df_targ, df_targ_wt, test_size=0.2, random_state=cnt)

#     clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=cnt, 
#                                  max_depth=4, 
#                                  min_samples_split=8)
    complexity_par = {'class_weight': 'balanced', 
                      'criterion': 'entropy', 
                      'max_depth': 2, 
                      'min_samples_split': 9, 
                      'oob_score': False}
    

    clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, **complexity_par)
    
    df_fit = clf.fit(df_feat_fill_train, df_targ_train, df_targ_wt_train)
#     df_fit = clf.fit(df_feat_fill_train, df_targ_train)
    df_pred = df_fit.predict(df_feat_fill_test)
    
#     print(roc_auc_score(df_targ_test, df_pred, sample_weight=df_targ_wt_test))
#     print(roc_auc_score(df_targ_test, df_pred))
    print(test_score(df_targ_test, df_pred, df_targ_wt_test))

In [43]:
roc_auc_score?

Signature: roc_auc_score(y_true, y_score, average='macro', sample_weight=None, max_fpr=None)
Docstring:
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC)
from prediction scores.

Note: this implementation is restricted to the binary classification task
or multilabel classification task in label indicator format.

Read more in the :ref:`User Guide <roc_metrics>`.

Parameters
----------
y_true : array, shape = [n_samples] or [n_samples, n_classes]
    True binary labels or binary label indicators.

y_score : array, shape = [n_samples] or [n_samples, n_classes]
    Target scores, can either be probability estimates of the positive
    class, confidence values, or non-thresholded measure of decisions
    (as returned by "decision_function" on some classifiers). For binary
    y_true, y_score is supposed to be the score of the class with greater
    label.

average : string, [None, 'micro', 'macro' (default), 'samples', 'weighted']
    If ``None``, the scores for each

### RF with AUC and no PCA

In [82]:
df_feat_fill.shape

(17573, 1123)

## Tune Parameters

### Leave one out CV to find model complexity

In [ ]:
%%time 


### Define len for subset for development

_dev_len = 14000


### Define feature and target data

X = df_feat_fill[:_dev_len].to_numpy()
y = df_targ[:_dev_len].to_numpy()


### Save number of splits for leave-one-out CV

loo = LeaveOneOut()
splits = loo.split(X)


### Grid of hyperparams to search

# n_estimators_par = [10, 50, 100, 200]
max_depth_par = range(1,2)
# min_samples_split_par = [5,7,9,11,14,17,20,30]
# min_samples_leaf_par = range(1,11)
criterion_par = ['gini', 'entropy']
class_weight_par = [None, 'balanced', 'balanced_subsample']
# oob_score_par = [True, False]

parameters = {
#               'n_estimators':n_estimators_par, 
              'max_depth':max_depth_par, 
#               'min_samples_split':min_samples_split_par, 
#               'min_samples_leaf':min_samples_leaf_par, 
              'criterion':criterion_par, 
              'class_weight':class_weight_par, 
#               'oob_score':oob_score_par}
                }

rfc = RandomForestClassifier(n_estimators=5, n_jobs=1)
clf = GridSearchCV(rfc, parameters, n_jobs=-1, scoring=prof_score, cv=splits)
clf.fit(X, y)

In [193]:
clf.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 2,
 'min_samples_split': 9,
 'oob_score': False}

In [194]:
clf.score(X, y)

0.5562666666666667

In [195]:
X_val = df_feat_fill[_dev_len:].to_numpy()
y_val = df_targ[_dev_len:].to_numpy()

clf_pred = clf.predict(X_val)
roc_auc_score(y_val, clf_pred)

0.5828358742850351

In [ ]:
df_feat_fill.shape

In [ ]:
X.shape

In [ ]:
for cnt in range(1,100,10):
#     X_train, X_test, Y_train, Y_test = train_test_split(df_feat_fill, df_targ, test_size=0.2, random_state=cnt)

    ### Standarize data

    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(df_feat_fill)
    df_feat_fill_st = pd.DataFrame(np_scaled)

    df_feat_fill_train, df_feat_fill_test, df_targ_train, df_targ_test = train_test_split(df_feat_fill_st, df_targ, test_size=0.2)

    clf = AdaBoostClassifier()
    df_fit = clf.fit(df_feat_fill_train, df_targ_train)
    df_pred = df_fit.predict(df_feat_fill_test)
    
    print(roc_auc_score(df_targ_test, df_pred))
    

## Test out PCA

### Standardize Features

In [75]:
def RunPCA(X,n):
    """Takes an input data set X and returns n principal components
    """
    # Create a scaler object
    sc = StandardScaler()
    
    # Fit the scaler to the features and transform
    X_std = sc.fit_transform(X)

    # Create a pca object with the 2 components as a parameter
    pca = decomposition.PCA(n_components=n)

    # Fit the PCA and transform the data
    X_std_pca = pca.fit_transform(X_std)
    
    return X_std_pca

In [76]:
df_feat_fill_train = df_feat_fill.iloc[:nsplit]
df_targ_train = df_targ.iloc[:nsplit]
df_feat_fill_test = df_feat_fill.iloc[nsplit:]
df_targ_test = df_targ.iloc[nsplit:]

### Practice with PCA

transform df with fit on train

### Tune n_components param

In [94]:
scores = dict()

for cnt in range(1,100,5):

    ### Create a pca object with the 2 components as a parameter
    
    pca = decomposition.PCA(n_components=50)

    df_feat_fill_train, df_feat_fill_test, df_targ_train, df_targ_test = train_test_split(df_feat_fill, df_targ, test_size=0.1, random_state=1)

    X1 = df_feat_fill_train
    X2 = df_feat_fill_test

    ### Create a scaler object
    
    sc = StandardScaler()

    ### Fit the scaler to the features and transform
    
    X1_std = sc.fit_transform(X1)
    X2_std = sc.fit(X1).transform(X2)

    ### Fit the PCA and transform the data
    
    X1_std_pca = pca.fit_transform(X1_std)
    X2_std_pca = pca.fit(X1_std).transform(X2_std)

    std_pca_train = X1_std_pca
    std_pca_df = X2_std_pca

    clf = AdaBoostClassifier(n_estimators=100, random_state=1)
    df_fit = clf.fit(std_pca_train, df_targ_train)
    df_pred = df_fit.predict(std_pca_df)
    score = roc_auc_score(df_targ_test, df_pred)
    
    ### Append score to dict
    
    scores[cnt] = score
    
    ### Print for OCD
    print(cnt,' ',score)

1   0.5406012163200662
6   0.5254810861839616
11   0.5257931631733549
16   0.5242600849629604
21   0.5194033868155274
26   0.5261052401627482
31   0.5394582343464133
36   0.5252879885467745
41   0.5288456662258579
46   0.530548436299235
51   0.512311437231565
56   0.5231171029893075
61   0.5362496927992135
66   0.5289373388414922
71   0.5312876686678604
76   0.5225846216261553
81   0.5338584028679876
86   0.5248198730626845
91   0.5148587656574877
96   0.531898169278361


In [ ]:
RandomForestClassifier?

In [ ]:
pd.DataFrame([clf.feature_importances_, df_feat_fill_train.columns]).transpose().sort_values

In [ ]:
pd.DataFrame([clf.feature_importances_, df_feat_fill_train.columns]).transpose.sort

In [ ]:
df_feat_fill_desc = df_feat_fill_train.describe().loc[['mean', 'std']]

In [ ]:
df_feat_fill_desc

In [ ]:
for x in df_feat_fill_desc.columns:
    print(df_feat_fill_desc[x])

In [ ]:
scores = dict()
for cnt in range(4):
    scores[cnt] = cnt*4

In [ ]:
scores